In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
address_file_book = './data/esteftaat-emam-khamenei.htm'

In [3]:
''''
    This method for opening a file, reading its data, and returning the read information.
'''
def read_books(address_file):
    return open(address_file, mode='r', encoding='utf-8').read()


# read file
book = read_books(address_file_book)

In [4]:
soup_content = BeautifulSoup(book, 'html.parser')

In [5]:
QA_Data = {}
def boutifulsoup_filtertags(book):
        h1 = None
        h2 = None
        h3 = None
        h4 = None
        soup_content = BeautifulSoup(book, 'html.parser')
        # return soup_content
        for soup in soup_content.descendants:
            if soup != None and soup.name != None:
                
                if 'h1' in soup.name:
                    h1 = soup.text
                    h2 = None
                    h3 = None
                    h4 = None
                    if not soup.text in QA_Data:
                        QA_Data[h1] = {}

                if 'h2' in soup.name:
                    h2 = soup.text
                    h3 = None
                    h4 = None
                    if not soup.text in QA_Data[h1]:
                        QA_Data[h1][h2] = {}

                if 'h3' in soup.name:
                    h3 = soup.text
                    h4 = None
                    if not soup.text in QA_Data[h1][h2]:
                        QA_Data[h1][h2][h3] = {}
                    
                if 'h4' in soup.name:
                    h4 = soup.text
                    if not soup.text in QA_Data[h1][h2][h3]:
                        QA_Data[h1][h2][h3][h4] = {}

                if h1 != None and h2 != None and h3 != None and h4 != None and soup != None and soup.text != None:
                    if soup.name == 'p':
                        text = soup.text
                        QA_Data[h1][h2][h3][h4][text] = {}
                        if soup.a != None:
                            QA_Data[h1][h2][h3][h4][text] = soup.a['title']
                else:
                    if h1 == None:
                        if soup.name == 'p':
                            text = soup.text
                            if not '...' in QA_Data:
                                QA_Data['...'] = {}
                            QA_Data['...'][text] = {}
                            if soup.a != None:
                                QA_Data['...'][text] = soup.a['title']
                    elif h2 == None:
                        if soup.name == 'p':
                            text = soup.text
                            if not '...' in QA_Data[h1]:
                                QA_Data[h1]['...'] = {}    
                            QA_Data[h1]['...'][text] = {}
                            if soup.a != None:
                                QA_Data[h1]['...'][text] = soup.a['title']
                    elif h3 == None:
                        if soup.name == 'p':
                            text = soup.text
                            if not '...' in QA_Data[h1][h2]:
                                QA_Data[h1][h2]['...'] = {}    
                            QA_Data[h1][h2]['...'][text] = {}
                            if soup.a != None:
                                QA_Data[h1][h2]['...'][text] = soup.a['title']
                    elif h4 == None:
                        if soup.name == 'p':
                            text = soup.text
                            if not '...' in QA_Data[h1][h2][h3]:
                                QA_Data[h1][h2][h3]['...'] = {}    
                            QA_Data[h1][h2][h3]['...'][text] = {}
                            if soup.a != None:
                                QA_Data[h1][h2][h3]['...'][text] = soup.a['title']
        return soup_content
soup = boutifulsoup_filtertags(book)

In [6]:
book = QA_Data['رساله اجوبة الاستفتائات مطابق با فتاواي حضرت آيه الله العظمي آقاي حاج سيّد على حسينى خامنه اى دام ظله العالى']
QAS = {}
QA_temp = []
for h3 in book:
    if not (h3 == 'مشخصات كتاب'):
        QA_temp = []
        for h4 in book[h3]:
            if not book[h3][h4]:
                QA_temp.append(h4)
            for h5 in book[h3][h4]:
                if not book[h3][h4][h5]:
                    QA_temp.append(h5)
                for h6 in book[h3][h4][h5]:
                    if not book[h3][h4][h5][h6]:
                        QA_temp.append(h6)
                    for h7 in book[h3][h4][h5][h6]:
                        if not book[h3][h4][h5][h6][h7]:
                            QA_temp.append(h7)
        QAS[h3] = QA_temp

In [7]:
Q_all = []
A_all = []


for label in QAS:
    Q = []
    A = []
    temp = []
    TTT = False  
    for i in QAS[label]:
        if re.search('س [0-9]*:', i) and (re.search('ج:', i) or re.search('ج\)', i)):
            index = i.rfind('ج:')
            Q.append(i[:index])
            A.append(i[index:])       
        else:
            if re.search('س [0-9]*:', i):
                TTT = True
                A.extend(temp)
                temp = []
            elif re.search('ج:', i) or re.search('ج\)', i):
                if not TTT:
                    Q.extend(temp)
                    temp = []
                TTT = False
            if TTT:
                Q.append(i)
            else:
                temp.append(i)
    Q_all.append(Q)
    A_all.append(A)
A = A[1:]

In [8]:
print(len(Q_all))
print(len(A_all))

80
80


In [9]:
A_all[0].pop(0)

'راههاى سه گانه: احتياط، اجتهاد، تقليد'

In [12]:
QQ = []
for j in Q_all:
    for i in j:
        if re.search('س [0-9]*:', i):
            QQ.append(i)
        else:
            QQ[-1] += i

In [13]:
AA = []
for j in A_all:
    for i in j:
        if re.search('ج:', i) or re.search('ج\)', i):
            AA.append(i)
        else:
            AA[-1] += i

In [14]:
len(QQ)

2157

In [15]:
len(AA)

2077

In [16]:
all_data = {'question':[], 'answer':[]}
for q, a in zip(QQ, AA):
    all_data['question'].append(q)
    all_data['answer'].append(a)

In [17]:
pd.DataFrame(all_data).to_csv('khamenei.csv')

In [26]:
pd.read_csv('khamenei.csv')[-35:]

,question,answer
2084,س 2087: اگر ملكى كه توسط مالك، حبس دائم بر چيز...,ج: مجرّد سند وقف اعم از اينكه اصلى باشد يا از ...
2085,NaN,ج: اگر اين كار بعد از انشاء وقف بر وجه شرعى و ...
2086,NaN,ج: وقف ايشان فقط در سهم خودشان از تركه صحيح اس...
2087,NaN,ج: مجرّد ثبت زمين مذكور به نام اولاد اناث، توس...
2088,NaN,ج: اگر اصل وقف بودن آن بر وجه شرعى ثابت شود جا...
2089,NaN,ج: تا زمانى كه ثابت نشود كه زمين آزمايشگاه برا...
2090,NaN,ج: انتفاع از وقف و تصرّف در آن هرچند به مصلحت ...
2091,NaN,ج: اگر ثابت شود كه انتفاع از آن كتابهاى وقفى ن...
2092,NaN,ج: بعد از ثبوت اصل وقف، تا زمانى كه ثابت نشده ...
2093,NaN,ج: وقف سهم ملك مشاع هرچند قبل از تفكيك باشد، ش...
